In [1]:
pwd

'D:\\Securing MULTIMODAL biometrics using FHE\\PAPER2 LATEST'

In [2]:
pip install torch torchvision insightface pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.2.2+cpu
False


In [4]:
!python --version


Python 3.8.19


In [5]:
pip install insightface==0.7.3

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install onnx==1.11.0

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install onnxruntime==1.10.0

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torch torchvision numpy pillow

Note: you may need to restart the kernel to use updated packages.


In [148]:
pwd

'D:\\Securing MULTIMODAL biometrics using FHE\\PAPER2 LATEST'

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from insightface.app import FaceAnalysis

# Initialize the ArcFace model using InsightFace
app = FaceAnalysis(allowed_modules=['detection', 'recognition'])
app.prepare(ctx_id=0, det_size=(320, 320))  # You can change det_size=(320, 320) for small images

# Directory paths
# DATA_DIR = r'D:\paper2\NEWCODE\ORL_Face_dataset_2image' 
# output_csv_path = 'D:/Securing MULTIMODAL biometrics using FHE/PAPER2 LATEST/Features/ORL_features_arcface_2img.csv'

DATA_DIR = r'/dataset-path'
output_csv_path = 'Features.csv'
# Initialize a list to store features and image names
features_list = []

# Process each image in the dataset
for class_dir in sorted(os.listdir(DATA_DIR)):
    class_path = os.path.join(DATA_DIR, class_dir)
    if os.path.isdir(class_path):
        for img_name in sorted(os.listdir(class_path)):
            if img_name.lower().endswith('.jpg'):
                img_path = os.path.join(class_path, img_name)
                print(f"Processing {img_path}...")

                # Load image and convert to RGB NumPy array
                try:
                    img = Image.open(img_path).convert('RGB')
                    img_np = np.array(img)

                    # Extract features using ArcFace
                    faces = app.get(img_np)
                    if len(faces) > 0:
                        feature_vector = faces[0].embedding.flatten()

                        # Construct image identifier and save features
                        img_identifier = f"{class_dir}/{img_name}"
                        features_list.append([img_identifier] + feature_vector.tolist())
                    else:
                        print(f"No face detected in {img_path}")
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

# Generate column names for the header
if features_list:
    num_features = len(features_list[0]) - 1  # Subtract 1 for 'Image Name' column
    column_names = ['Image Name'] + [f'f{i+1}' for i in range(num_features)]

    # Convert to a DataFrame and save to CSV
    df = pd.DataFrame(features_list, columns=column_names)
    df.to_csv(output_csv_path, index=False)
    print(f"\nFeature extraction completed. Features saved to:\n{output_csv_path}")
else:
    print("No features were extracted. Please check your input images.")


In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import eigh

# Load the face features from the CSV file
csv_path = 'features.csv'
df = pd.read_csv(csv_path)

# Extract the feature vectors (ignoring the first column with image names)
X = df.iloc[:, 1:].values

# LPP function
def compute_weight_matrix(X, epsilon):
    distances = squareform(pdist(X.T, 'euclidean'))
    W = np.zeros_like(distances)
    W[distances <= epsilon] = 1
    return W

def compute_laplacian_matrix(W):
    D = np.diag(np.sum(W, axis=1))
    L = D - W
    return L

def lpp(X, num_components, epsilon):
    W = compute_weight_matrix(X, epsilon)
    L = compute_laplacian_matrix(W)
    D = np.diag(np.sum(W, axis=1))
    
    # Eigen decomposition of the generalized eigenvalue problem
    eigvals, eigvecs = eigh(L, D)
    
    # Sort the eigenvectors by eigenvalues
    sorted_indices = np.argsort(eigvals)
    eigvecs = eigvecs[:, sorted_indices]
    
    # Select the top `num_components` eigenvectors
    V = eigvecs[:, :num_components]
    
    # Project the original data using the selected eigenvectors
    X_reduced = np.dot(X, V)  
    return X_reduced

# Parameters for LPP
num_components = 256  # Reduce to 256 dimensions
epsilon = 0.5        

# Apply LPP
X_reduced = lpp(X, num_components, epsilon)

# Create a DataFrame to save the reduced features
reduced_df = pd.DataFrame(X_reduced, columns=[f'f{i+1}' for i in range(num_components)])

# Insert the image names back into the DataFrame
reduced_df.insert(0, 'Image Name', df['Image Name'])

# Save the reduced features to a new CSV file
output_path = 'features_dim.csv'
reduced_df.to_csv(output_path, index=False)

print("LPP feature reduction completed and saved to:", output_path)


In [89]:
# Find Genuine and Impostor scores

In [ ]:
import os
import csv
import numpy as np
from scipy.spatial.distance import euclidean
from itertools import combinations

# Load the extracted features
def load_features(csv_filename):
    features = {}
    with open(csv_filename, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        headers = next(csv_reader)  # Skip header
        for row in csv_reader:
            image_name = row[0]
            feature_vector = np.array(row[1:], dtype=float)
            features[image_name] = feature_vector
    return features

# Calculate Euclidean distance and classify pairs
def classify_pairs(features):
    genuine_pairs = []
    impostor_pairs = []
    
    all_pairs = list(combinations(features.keys(), 2))
    
    for (image1, image2) in all_pairs:
        class1 = image1.split('/')[0]
        class2 = image2.split('/')[0]
        dist = euclidean(features[image1], features[image2])
        
        if class1 == class2:
            genuine_pairs.append((image1, image2, dist))
        else:
            impostor_pairs.append((image1, image2, dist))
    
    return genuine_pairs, impostor_pairs

# Save pairs to CSV files
def save_pairs_to_csv(pairs, filename):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Image Pair', 'Distance'])
        for (image1, image2, dist) in pairs:
            csv_writer.writerow([f"{image1} - {image2}", dist])

features_filename = 'features-dim.csv'


genuine_output_filename = 'genuine-arcface.csv'
impostor_output_filename = 'impostor-arcface.csv'

# Load features
features = load_features(features_filename)

# Classify pairs
genuine_pairs, impostor_pairs = classify_pairs(features)

# Save pairs to CSV files
save_pairs_to_csv(genuine_pairs, genuine_output_filename)
save_pairs_to_csv(impostor_pairs, impostor_output_filename)

# Print the count of genuine and impostor matches
print(f"Number of genuine matches: {len(genuine_pairs)}")
print(f"Number of impostor matches: {len(impostor_pairs)}")

print("Genuine and impostor matches calculation and saving complete.")


In [109]:
# find and plot FAR , FRR and EER

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# Load distances from CSV files
def load_distances(csv_filename):
    distances = []
    with open(csv_filename, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)  # Skip header
        for row in csv_reader:
            dist = float(row[1])
            distances.append(dist)
    return distances

# Calculate FAR, FRR, and EER
def calculate_far_frr_eer(genuine_distances, impostor_distances):
    genuine_distances = np.array(genuine_distances)
    impostor_distances = np.array(impostor_distances)
    
    min_threshold = min(np.min(genuine_distances), np.min(impostor_distances))
    max_threshold = max(np.max(genuine_distances), np.max(impostor_distances))
    
    thresholds = np.linspace(min_threshold, max_threshold, num=1000)
    
    frr = np.zeros(len(thresholds))
    far = np.zeros(len(thresholds))

    for i, threshold in enumerate(thresholds):
        frr[i] = np.sum(genuine_distances > threshold) / len(genuine_distances)
        far[i] = np.sum(impostor_distances <= threshold) / len(impostor_distances)
        
    eer_index = np.abs(frr - far).argmin()
    eer = np.mean((frr[eer_index], far[eer_index]))
    
    return far, frr, eer, thresholds[eer_index], thresholds

# File paths
genuine_output_filename = 'genuine-arcface-dim.csv'
impostor_output_filename = 'genuine-arcface-dim.csv'

# Load distances from CSV files
genuine_distances = load_distances(genuine_output_filename)
impostor_distances = load_distances(impostor_output_filename)

# Calculate FAR, FRR, and EER
far, frr, eer, eer_threshold, thresholds = calculate_far_frr_eer(genuine_distances, impostor_distances)

# Print results
print(f"EER: {eer}")
print(f"Threshold at EER: {eer_threshold}")

# Plot FAR and FRR
plt.figure(figsize=(10, 6))
plt.plot(thresholds, far, label='FAR', color='red')
plt.plot(thresholds, frr, label='FRR', color='blue')
plt.axvline(x=eer_threshold, linestyle='--', color='green', label=f'EER Threshold: {eer_threshold:.4f}')
plt.axhline(y=eer, linestyle='--', color='purple', label=f'EER: {eer:.4f}')
plt.xlabel('Threshold')
plt.ylabel('Rate')
plt.title('FAR and FRR vs. Threshold')
plt.legend()
plt.grid(True)

# Save the plot
output_plot_filename = 'FAR-FRR.png'
plt.savefig(output_plot_filename)

print(f"Plot saved as {output_plot_filename}")

plt.show()


In [111]:
def calculate_confusion_matrix(genuine_distances, impostor_distances, threshold):
    # Convert lists to NumPy arrays
    genuine_distances = np.array(genuine_distances)
    impostor_distances = np.array(impostor_distances)
    
    tp = np.sum(genuine_distances <= threshold)  # True Positives
    fn = np.sum(genuine_distances > threshold)   # False Negatives
    tn = np.sum(impostor_distances > threshold)  # True Negatives
    fp = np.sum(impostor_distances <= threshold) # False Positives
    
    return tp, tn, fp, fn

def calculate_accuracy(tp, tn, fp, fn):
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy

# Assuming you've already calculated the threshold at EER using the existing code
eer_threshold = 15.42
 

# Call the confusion matrix function
tp, tn, fp, fn = calculate_confusion_matrix(genuine_distances, impostor_distances, eer_threshold)

# Calculate accuracy
accuracy = calculate_accuracy(tp, tn, fp, fn)

# Print results
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"Accuracy: {accuracy:.4f}")


True Positives (TP): 41
True Negatives (TN): 3280
False Positives (FP): 0
False Negatives (FN): 0
Accuracy: 1.0000


In [112]:
def calculate_precision(tp, fp):
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def calculate_recall(tp, fn):
    return tp / (tp + fn) if (tp + fn) > 0 else 0

def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Call the confusion matrix function
tp, tn, fp, fn = calculate_confusion_matrix(genuine_distances, impostor_distances, eer_threshold)

# Calculate accuracy
accuracy = calculate_accuracy(tp, tn, fp, fn)

# Calculate precision, recall, and F1 score
precision = calculate_precision(tp, fp)
recall = calculate_recall(tp, fn)
f1_score = calculate_f1_score(precision, recall)

# Print results
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


True Positives (TP): 41
True Negatives (TN): 3280
False Positives (FP): 0
False Negatives (FN): 0
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


In [125]:
# Encryption

In [126]:
# Generating the Keys
import utilities
import tenseal as ts
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes = [60, 40, 40,60]
)

context.generate_galois_keys()
context.global_scale=2**40

secret_context = context.serialize(save_secret_key = True)
utilities.write_data("/secret_key.txt",secret_context) 

context.make_context_public() # drops private key
public_context=context.serialize()
utilities.write_data("/public_key.txt",public_context)

In [ ]:
import os
import numpy as np
import pandas as pd

def extract_embeddings_from_csv(csv_file, embedding_folder):
    # Load the feature vectors from the CSV file
    df = pd.read_csv(csv_file)
    image_paths = df['Image Name']
    feature_vectors = df.iloc[:, 1:].values  # Extract all feature columns

    # Create the embedding folder if it doesn't exist
    if not os.path.exists(embedding_folder):
        os.makedirs(embedding_folder)
        
    # Count the number of embeddings extracted
    total_embeddings = 0

    # Iterate over the feature vectors and save the embeddings
    for i in range(len(image_paths)):
        image_path = image_paths[i]
        embedding = feature_vectors[i]

        # Extract the folder name from the image path
        folder_name = os.path.basename(os.path.dirname(image_path))

        # Create the folder structure in the embedding folder
        embedding_subfolder = os.path.join(embedding_folder, folder_name)
        os.makedirs(embedding_subfolder, exist_ok=True)

        # Save the embedding as a numpy file
        embedding_file = os.path.join(embedding_subfolder, os.path.splitext(os.path.basename(image_path))[0] + '.npy')
        np.save(embedding_file, embedding)
        
        # Increment the count
        total_embeddings += 1

    print("Embeddings calculated and saved successfully!")
    
    return total_embeddings

# Set the CSV file
csv_file = 'arcface-features.csv'

 
# Set the embedding folder

embedding_folder = '/ORL_arcface_Embedding'

# Extract embeddings and count the total number of embeddings
total_embeddings = extract_embeddings_from_csv(csv_file, embedding_folder)

# Print the total number of embeddings
print("Total number of embeddings:", total_embeddings)


In [ ]:
import os
import numpy as np
from pathlib import Path
import tenseal as ts

def encrypt_embedding(embedding_file, embedding_folder, encrypted_folder):
    context = ts.context_from(utilities.read_data("/secret_key.txt"))  
    embedding = np.load(embedding_file)
    embedding = embedding.flatten()  # Flatten the multi-dimensional array to a 1-dimensional vector
    enc_embedding = ts.ckks_vector(context, embedding)
    enc_embedding_proto = enc_embedding.serialize()
    
    # Create the folder structure in the encrypted folder
    relative_folder = os.path.relpath(os.path.dirname(embedding_file), embedding_folder)
    encrypted_embedding_folder = os.path.join(encrypted_folder, relative_folder)
    os.makedirs(encrypted_embedding_folder, exist_ok=True)
    
    # Store the encrypted embedding in the corresponding folder
    enc_embedding_file = os.path.join(encrypted_embedding_folder, os.path.basename(embedding_file) + ".txt")
    utilities.write_data(enc_embedding_file, enc_embedding_proto)
    
    del context, enc_embedding, enc_embedding_proto
    return enc_embedding_file

# Set the embedding folders 

embedding_folder = '/ORL_arcface_Embeddings'


# Set the encrypted embedding folders
encrypted_folder = '/ORL_arcface_Encrypted-Embeddings'


# Iterate over the embeddings in the train embedding folder and encrypt them
for root, dirs, files in os.walk(embedding_folder):
    for file in files:
        embedding_file = os.path.join(root, file)
        encrypt_embedding(embedding_file, embedding_folder, encrypted_folder)



print("Encrypted embeddings generated and stored successfully!")


In [ ]:
import os
import numpy as np
import tenseal as ts
import time

def encrypt_embedding(embedding_file, embedding_folder, encrypted_folder):
    # Start timing for the encryption
    start_time = time.time()
    
    # Load context and embedding, then perform encryption
    context = ts.context_from(utilities.read_data("/secret_key.txt"))  
    embedding = np.load(embedding_file)
    embedding = embedding.flatten()  # Flatten the multi-dimensional array to a 1-dimensional vector
    enc_embedding = ts.ckks_vector(context, embedding)
    enc_embedding_proto = enc_embedding.serialize()
    
    # Create the folder structure in the encrypted folder
    relative_folder = os.path.relpath(os.path.dirname(embedding_file), embedding_folder)
    encrypted_embedding_folder = os.path.join(encrypted_folder, relative_folder)
    os.makedirs(encrypted_embedding_folder, exist_ok=True)
    
    # Store the encrypted embedding in the corresponding folder
    enc_embedding_file = os.path.join(encrypted_embedding_folder, os.path.basename(embedding_file) + ".txt")
    utilities.write_data(enc_embedding_file, enc_embedding_proto)
    
    # Clean up
    del context, enc_embedding, enc_embedding_proto
    
    # Calculate the elapsed time for encryption
    end_time = time.time()
    encryption_time = end_time - start_time
    
    # Return the path to the encrypted embedding and the computation time
    return enc_embedding_file, encryption_time

# Specify the embedding and encrypted embedding folders

embedding_folder = '/ORL_arcface_Embeddings'
encrypted_folder = '/ORL_arcface_Encrypted-Embeddings'


# Select a single embedding file for testing (change this to the specific file path if needed)
test_embedding_file = next(os.path.join(root, file) 
                           for root, _, files in os.walk(embedding_folder) for file in files)

# Encrypt the single embedding and get the computation time
enc_file, encryption_time = encrypt_embedding(test_embedding_file, embedding_folder, encrypted_folder)

# Print the computation time for this single embedding encryption
print(f"Encrypted {os.path.basename(test_embedding_file)} in {encryption_time:.3f} seconds")
print(f"Encrypted file saved to: {enc_file}")


In [131]:
# Function to calculate the Euclidean distance between two encrypted embeddings
def calculate_distance(enc_embedding_file1, enc_embedding_file2):
    context = ts.context_from(utilities.read_data("/public_key.txt"))
    enc_embedding_proto1 = utilities.read_data(enc_embedding_file1)
    enc_embedding_proto2 = utilities.read_data(enc_embedding_file2)
    enc_embedding1 = ts.lazy_ckks_vector_from(enc_embedding_proto1)
    enc_embedding1.link_context(context)
    enc_embedding2 = ts.lazy_ckks_vector_from(enc_embedding_proto2)
    enc_embedding2.link_context(context)
    euclidean_squared = enc_embedding1 - enc_embedding2
    euclidean_squared = euclidean_squared.dot(euclidean_squared)
    euclidean_squared_file = os.path.join(euclidean_folder, os.path.basename(enc_embedding_file1) + "_-_" +
                                          os.path.basename(enc_embedding_file2) + ".txt")
    utilities.write_data(euclidean_squared_file, euclidean_squared.serialize())
    return euclidean_squared_file

# Function to decrypt and compare the Euclidean distance
def decrypt_and_compare(euclidean_squared_file):
    if euclidean_squared_file is None:
        return None  # Skip processing if the file path is None

    context = ts.context_from(utilities.read_data("/secret_key.txt"))
    euclidean_squared_proto = utilities.read_data(euclidean_squared_file)
    euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
    euclidean_squared.link_context(context)
    euclidean_squared_plain = euclidean_squared.decrypt()[0]
    return euclidean_squared_plain


In [132]:
# Genuine-Distance Calculation

In [ ]:
import os
import glob
import tenseal as ts
import utilities

def calculate_distance(enc_embedding_file1, enc_embedding_file2):
    context = ts.context_from(utilities.read_data("/public_key.txt"))
    enc_embedding_proto1 = utilities.read_data(enc_embedding_file1)
    enc_embedding_proto2 = utilities.read_data(enc_embedding_file2)
    enc_embedding1 = ts.lazy_ckks_vector_from(enc_embedding_proto1)
    enc_embedding1.link_context(context)
    enc_embedding2 = ts.lazy_ckks_vector_from(enc_embedding_proto2)
    enc_embedding2.link_context(context)

    # Calculate the squared Euclidean distance between the two encrypted vectors
    euclidean_squared = enc_embedding1 - enc_embedding2
    euclidean_squared = euclidean_squared.dot(euclidean_squared)

    # Save the encrypted Euclidean distance result in a file
    euclidean_squared_file = os.path.join(euclidean_folder, f"{os.path.basename(enc_embedding_file1)}_-_{os.path.basename(enc_embedding_file2)}.txt")
    utilities.write_data(euclidean_squared_file, euclidean_squared.serialize())
    
    return euclidean_squared_file


# Set the folder paths

embedding_folder = '/ORL_arcface_Embeddings'
euclidean_folder = '/ORL_arcface_Genuine-Euclidean-Distances'


# Create the Euclidean distance folder if it doesn't exist
os.makedirs(euclidean_folder, exist_ok=True)

# Get the list of class folders inside the encrypted embeddings folder
class_folders = glob.glob(os.path.join(encrypted_folder, '*'))

# Iterate over each class folder
for class_folder in class_folders:
    # Get the list of embedding files in the current class folder
    embedding_files = glob.glob(os.path.join(class_folder, '*.txt'))

    # Ensure that there are exactly 2 embeddings in each class folder for genuine distance calculation
    if len(embedding_files) == 2:
        # Calculate the Euclidean distance between the two embeddings
        euclidean_squared_file = calculate_distance(embedding_files[0], embedding_files[1])

        # Extract image names for logging
        image1_name = os.path.splitext(os.path.basename(embedding_files[0]))[0]
        image2_name = os.path.splitext(os.path.basename(embedding_files[1]))[0]

        # Print the results to the console
        print(f"Class: {os.path.basename(class_folder)}, Image 1: {image1_name}, Image 2: {image2_name}, Euclidean Distance File: {euclidean_squared_file}")

print("Genuine distance calculation complete. Encrypted distances are stored in the specified folder.")


In [135]:
# Decrypt the genuine distances and store them in a csv file 

In [ ]:
import os
import glob
import tenseal as ts
import utilities
import csv

def decrypt_and_generate_distances(euclidean_folder):
    decrypted_distances = []
    
    # Loop through each encrypted file in the specified folder
    for encrypted_file in glob.glob(os.path.join(euclidean_folder, '*.txt')):
        try:
            # Load the encryption context using the secret key
            context = ts.context_from(utilities.read_data("/secret_key.txt"))
            
            # Load the encrypted distance data
            encrypted_distance_proto = utilities.read_data(encrypted_file)
            encrypted_distance = ts.lazy_ckks_vector_from(encrypted_distance_proto)
            encrypted_distance.link_context(context)
            
            # Decrypt the distance and extract the first value (scalar)
            decrypted_distance = encrypted_distance.decrypt()[0]
            
            # Append the decrypted distance to the list
            decrypted_distances.append(decrypted_distance)
        except Exception as e:
            # Handle decryption errors gracefully
            print(f"Error decrypting {encrypted_file}: {str(e)}")

    # Define the path for the output CSV file
    csv_file = os.path.join(euclidean_folder, "/ORL_arcface_Genuine-Euclidean-Distances.csv")
    
    # Write the decrypted distances to a CSV file
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Distance"])  # CSV header
        writer.writerows([[distance] for distance in decrypted_distances])  # Write each distance

    print(f"Decryption and distance generation completed. Total distances: {len(decrypted_distances)}")

# Set the folder containing the encrypted distance files
# euclidean_folder = 'D:/Securing MULTIMODAL biometrics using FHE/PAPER2 LATEST/FHE/arcface_Genuine-Euclidean-Distances_LPP16_FHE'

euclidean_folder = '/ORL_arcface_Genuine-Euclidean-Distances'

# Decrypt and generate distances
decrypt_and_generate_distances(euclidean_folder)


In [138]:
import os
import glob
import tenseal as ts
import utilities
import csv

def decrypt_and_generate_distances(euclidean_folder):
    decrypted_distances = []
    
    # Loop through each encrypted file in the specified folder
    for encrypted_file in glob.glob(os.path.join(euclidean_folder, '*.txt')):
        try:
            # Load the encryption context using the secret key
            context = ts.context_from(utilities.read_data("/secret_key.txt"))
            
            # Load the encrypted distance data
            encrypted_distance_proto = utilities.read_data(encrypted_file)
            encrypted_distance = ts.lazy_ckks_vector_from(encrypted_distance_proto)
            encrypted_distance.link_context(context)
            
            # Decrypt the distance and extract the first value (scalar)
            decrypted_distance = encrypted_distance.decrypt()[0]
            
            # Append the decrypted distance to the list
            decrypted_distances.append(decrypted_distance)
        except Exception as e:
            # Handle decryption errors gracefully
            print(f"Error decrypting {encrypted_file}: {str(e)}")

    # Define the path for the output CSV file
    
    #csv_file = 'D:/Securing MULTIMODAL biometrics using FHE/FHE/arcFace-Genuine-Distances_without_Lpp.csv'
    csv_file = '/ORL_arcface_Genuine-Euclidean-Distances.csv'
    
    # Write the decrypted distances to a CSV file
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Distance"])  # CSV header
        writer.writerows([[distance] for distance in decrypted_distances])  # Write each distance

    print(f"Decryption and distance generation completed. Total distances: {len(decrypted_distances)}")

# Set the folder containing the encrypted distance files
euclidean_folder = '/ORL_arcface_Genuine-Euclidean-Distances'

# Decrypt and generate distances
decrypt_and_generate_distances(euclidean_folder)


Decryption and distance generation completed. Total distances: 41


In [139]:
# IMpostor Calculation

In [146]:
import os
import glob
import tenseal as ts
import utilities
import csv

def decrypt_and_generate_distances(euclidean_folder):
    decrypted_distances = []
    
    # Loop through each encrypted file in the specified folder
    for encrypted_file in glob.glob(os.path.join(euclidean_folder, '*.txt')):
        try:
            # Load the encryption context using the secret key
            context = ts.context_from(utilities.read_data("/secret_key.txt"))
            
            # Load the encrypted distance data
            encrypted_distance_proto = utilities.read_data(encrypted_file)
            encrypted_distance = ts.lazy_ckks_vector_from(encrypted_distance_proto)
            encrypted_distance.link_context(context)
            
            # Decrypt the distance and extract the first value (scalar)
            decrypted_distance = encrypted_distance.decrypt()[0]
            
            # Append the decrypted distance to the list
            decrypted_distances.append(decrypted_distance)
        except Exception as e:
            # Handle decryption errors gracefully
            print(f"Error decrypting {encrypted_file}: {str(e)}")

    # Define the path for the output CSV file
    
    csv_file = '/ORL_arcface_Impostor-Euclidean-Distances.csv'
    
    # Write the decrypted distances to a CSV file
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Distance"])  # CSV header
        writer.writerows([[distance] for distance in decrypted_distances])  # Write each distance

    print(f"Decryption and distance generation completed. Total distances: {len(decrypted_distances)}")

# Set the folder containing the encrypted distance files

euclidean_folder = '/ORL_arcface_Impostor-Euclidean-Distances'

# Decrypt and generate distances
decrypt_and_generate_distances(euclidean_folder)


Decryption and distance generation completed. Total distances: 3280


In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# Load distances from CSV files
def load_distances(csv_filename):
    distances = []
    with open(csv_filename, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)  # Skip header
        for row in csv_reader:
            dist = float(row[0])
            distances.append(dist)
    return distances

# Calculate FAR, FRR, and EER
def calculate_far_frr_eer(genuine_distances, impostor_distances):
    genuine_distances = np.array(genuine_distances)
    impostor_distances = np.array(impostor_distances)
    
    min_threshold = min(np.min(genuine_distances), np.min(impostor_distances))
    max_threshold = max(np.max(genuine_distances), np.max(impostor_distances))
    
    thresholds = np.linspace(min_threshold, max_threshold, num=1000)
    
    frr = np.zeros(len(thresholds))
    far = np.zeros(len(thresholds))

    for i, threshold in enumerate(thresholds):
        frr[i] = np.sum(genuine_distances > threshold) / len(genuine_distances)
        far[i] = np.sum(impostor_distances <= threshold) / len(impostor_distances)
        
    eer_index = np.abs(frr - far).argmin()
    eer = np.mean((frr[eer_index], far[eer_index]))
    
    return far, frr, eer, thresholds[eer_index], thresholds

# File paths

genuine_output_filename = '/ORL_arcface_Genuine-Euclidean-Distances.csv'
impostor_output_filename = '/ORL_arcface_Impostor-Euclidean-Distances.csv'




# Load distances from CSV files
genuine_distances = load_distances(genuine_output_filename)
impostor_distances = load_distances(impostor_output_filename)

# Calculate FAR, FRR, and EER
far, frr, eer, eer_threshold, thresholds = calculate_far_frr_eer(genuine_distances, impostor_distances)

# Print results
print(f"EER: {eer}")
print(f"Threshold at EER: {eer_threshold}")

# Plot FAR and FRR
plt.figure(figsize=(10, 6))
plt.plot(thresholds, far, label='FAR', color='red')
plt.plot(thresholds, frr, label='FRR', color='blue')
plt.axvline(x=eer_threshold, linestyle='--', color='green', label=f'EER Threshold: {eer_threshold:.4f}')
plt.axhline(y=eer, linestyle='--', color='purple', label=f'EER: {eer:.4f}')
plt.xlabel('Threshold')
plt.ylabel('Rate')
plt.title('FAR and FRR vs. Threshold')
plt.legend()
plt.grid(True)

# Save the plot
output_plot_filename = '/ORL_arcface_euclidean_EER.png'

plt.savefig(output_plot_filename)

print(f"Plot saved as {output_plot_filename}")

plt.show()
